In [1]:
!pip install nhl-api-py

In [5]:
!sudo apt-get update
!sudo apt-get install -y ocl-icd-libopencl1-opencl-headers clinfo
!sudo apt-get install -y libnvidia-compute-550

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,788 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,241 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Package

In [2]:
from nhlpy import NHLClient
import pandas as pd
import pyarrow.parquet as pq


In [3]:
processed2023 = pd.read_parquet('/content/drive/MyDrive/projects/nhl/data/processed/processed_shots_2023.parquet', engine='pyarrow')
processed2024 = pd.read_parquet('/content/drive/MyDrive/projects/nhl/data/processed/processed_shots_2024.parquet', engine='pyarrow')
processed2025 = pd.read_parquet('/content/drive/MyDrive/projects/nhl/data/processed/processed_shots_2025.parquet', engine='pyarrow')

In [4]:
full_df = pd.concat([processed2023, processed2024, processed2025])
full_df.head()

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,shot_class,position,shooter_hand,shooting_pct,glove_hand,save_pct,angle,shot_on_glove,situation,target
0,Home,giveaway,No rebound,Rush,5,5,77,6,wrist,O,shot-on-goal,C,L,0.0880,L,0.915213,26.57,LL,EV,0
1,Home,hit,No rebound,No rush,5,5,30,30,slap,O,shot-on-goal,D,R,0.0427,L,0.915213,26.95,RL,EV,0
2,Away,shot-on-goal,No rebound,No rush,5,5,35,30,wrist,O,shot-on-goal,D,L,0.0439,L,0.917598,29.05,LL,EV,0
3,Home,hit,No rebound,No rush,5,5,41,14,wrist,O,shot-on-goal,R,R,0.1086,L,0.915213,16.26,RL,EV,0
4,Home,hit,No rebound,No rush,5,5,46,17,wrist,O,shot-on-goal,R,R,0.1086,L,0.915213,21.57,RL,EV,0


In [6]:
!clinfo

Number of platforms                               1
  Platform Name                                   NVIDIA CUDA
  Platform Vendor                                 NVIDIA Corporation
  Platform Version                                OpenCL 3.0 CUDA 12.4.89
  Platform Profile                                FULL_PROFILE
  Platform Extensions                             cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_khr_fp64 cl_khr_3d_image_writes cl_khr_byte_addressable_store cl_khr_icd cl_nv_compiler_options cl_nv_device_attribute_query cl_nv_pragma_unroll cl_nv_copy_opts cl_khr_gl_event cl_nv_create_buffer cl_khr_int64_base_atomics cl_khr_int64_extended_atomics cl_nv_kernel_attribute cl_khr_device_uuid cl_khr_pci_bus_info cl_khr_external_semaphore cl_khr_external_memory cl_khr_external_semaphore_opaque_fd cl_khr_external_memory_opaque_fd
  Platform Extensions with Version                cl_khr

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [28]:
cleaned_df = full_df.dropna()
X = cleaned_df.drop(['target', 'shot_class'], axis=1)
y = cleaned_df['target']

In [29]:
X.head()

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,position,shooter_hand,shooting_pct,glove_hand,save_pct,angle,shot_on_glove,situation
0,Home,giveaway,No rebound,Rush,5,5,77,6,wrist,O,C,L,0.0880,L,0.915213,26.57,LL,EV
1,Home,hit,No rebound,No rush,5,5,30,30,slap,O,D,R,0.0427,L,0.915213,26.95,RL,EV
2,Away,shot-on-goal,No rebound,No rush,5,5,35,30,wrist,O,D,L,0.0439,L,0.917598,29.05,LL,EV
3,Home,hit,No rebound,No rush,5,5,41,14,wrist,O,R,R,0.1086,L,0.915213,16.26,RL,EV
4,Home,hit,No rebound,No rush,5,5,46,17,wrist,O,R,R,0.1086,L,0.915213,21.57,RL,EV


In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 683349 entries, 0 to 198630
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   home           683349 non-null  object 
 1   last_play      683349 non-null  object 
 2   rebound        683349 non-null  object 
 3   rush           683349 non-null  object 
 4   home_skaters   683349 non-null  int64  
 5   away_skaters   683349 non-null  int64  
 6   x_coord        683349 non-null  int64  
 7   y_coord        683349 non-null  int64  
 8   shot_type      683349 non-null  object 
 9   zone           683349 non-null  object 
 10  position       683349 non-null  object 
 11  shooter_hand   683349 non-null  object 
 12  shooting_pct   683349 non-null  float64
 13  glove_hand     683349 non-null  object 
 14  save_pct       683349 non-null  float64
 15  angle          683349 non-null  float64
 16  shot_on_glove  683349 non-null  object 
 17  situation      683349 non-null  ob

In [31]:
X.isna().sum()

,0
home,0
last_play,0
rebound,0
rush,0
home_skaters,0
away_skaters,0
x_coord,0
y_coord,0
shot_type,0
zone,0


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer

cat_pipeline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
num_pipeline = make_pipeline(StandardScaler(), MinMaxScaler((-1,1)))

preprocessing = ColumnTransformer([
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=num_pipeline)

In [33]:
log_reg = make_pipeline(preprocessing, LogisticRegression())
log_reg.fit(X_train, y_train)
goal_predictions = log_reg.predict(X_train)
goal_predictions[:5].round(-2)
y.iloc[:5].values

array([0, 0, 0, 0, 0])

In [34]:
probabilities_log = log_reg.predict_proba(X_test)
pred_test = log_reg.predict(X_test)

In [35]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

auc_test = roc_auc_score(y_test, probabilities_log[:,1])
auc_train = roc_auc_score(y_train, log_reg.predict_proba(X_train)[:,1])

confusion_matrix(y_test, pred_test)
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.93      1.00      0.97    127516
           1       0.88      0.00      0.00      9154

    accuracy                           0.93    136670
   macro avg       0.90      0.50      0.48    136670
weighted avg       0.93      0.93      0.90    136670



In [36]:
auc_test

np.float64(0.7004215724443305)

In [40]:
import joblib

joblib.dump(log_reg, '/content/drive/MyDrive/projects/nhl/models/log_reg_v1.pkl')

['/content/drive/MyDrive/projects/nhl/models/log_reg_v1.pkl']